# Take-Home assignment
## Kashif Sohail - Data Engineer 
Github: https://github.com/kashif-se/postpay-assignment <br>
LinkedIn: https://www.linkedin.com/in/kshfse/

### Imports and loading Data

In [1]:
import pandas as pd
from datetime import timedelta

In [2]:
df=pd.read_csv("interview_df.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   order_id      1000 non-null   object 
 1   customer_id   1000 non-null   object 
 2   created_date  1000 non-null   object 
 3   order_amount  1000 non-null   float64
 4   shop_id       1000 non-null   object 
 5   order_status  1000 non-null   object 
dtypes: float64(1), object(5)
memory usage: 47.0+ KB


Truncating `created_date` to `date` to aovid the hours difference when looking back for 7 days or 90 days

In [4]:
df["created_date"]=pd.to_datetime(df["created_date"])
df["created_date"]=df["created_date"].apply(lambda x: x.date())
df["created_date"]=pd.to_datetime(df["created_date"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_id      1000 non-null   object        
 1   customer_id   1000 non-null   object        
 2   created_date  1000 non-null   datetime64[ns]
 3   order_amount  1000 non-null   float64       
 4   shop_id       1000 non-null   object        
 5   order_status  1000 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 47.0+ KB


In [5]:
df

,order_id,customer_id,created_date,order_amount,shop_id,order_status
0,ea44f92ee05880ccb9d1eac565c781c6,124e7b421093cb5ae18df8256ea591ca,2020-01-01,1839.20,4716486e2f5489e28a43354320c656a4,paid
1,6ea88eb8ff98a78e07611a4e955a7d46,ba108833b7381247b6d20fea0501b8dc,2020-04-03,2302.25,4716486e2f5489e28a43354320c656a4,paid
2,342e47f90a085011c3afa94475dc61bd,6536928d8ecbd7d731f5fc2aa22573a9,2020-04-12,1120.06,4716486e2f5489e28a43354320c656a4,paid
3,9566725f601cc9d26943b55aaaf1e5b0,8dd13ca4b535b996a775b66f274b0c1f,2020-05-04,565.00,556e232c4549000a0bdd7e9d23860a86,paid
4,27309cda8bf3a5a17cc7d1781ff998bb,8463b19013300bff71bfa8101773ad3d,2020-06-29,2467.00,24758fef3aa92a7b71c759e248557886,paid
...,...,...,...,...,...,...
995,678abfea3e904a321ee8a0457e79189b,42e45fe7bfc70e571261179918805cdb,2022-01-04,73.00,8312706086e548c51cc2cfd9c6a010a1,due
996,70c87363189e507cc7a41abacc2d573a,91565ec5169fac1639aa0160f1340523,2022-01-04,3567.00,f7b3fb70d1663504b4c1ec5b1c095888,due
997,d355bff8791ec22b8a73ce20df148caa,d270ec89c34859033f70604017107709,2022-01-04,65.00,8312706086e548c51cc2cfd9c6a010a1,due
998,92c0338455e56a624cf7bb0b3bdaa400,d7052b67b357eb15ef0429df9ce47105,2022-01-05,402.00,14a206b5298dbfa9454e85293881a249,due


### Checking null values

In [6]:
df.isna().sum()

order_id        0
customer_id     0
created_date    0
order_amount    0
shop_id         0
order_status    0
dtype: int64

### EDA to get high level overview

In [7]:
order_status=list(df.order_status.unique())
print("Unique Order Status: ",order_status)

Unique Order Status:  ['paid', 'unpaid', 'due']


In [8]:
print(f"Create date range: {df.created_date.min()} till {df.created_date.max()}")

Create date range: 2020-01-01 00:00:00 till 2022-01-05 00:00:00


### Introducing new columns to calculate

In [9]:
df["orders_on_customer_id_7D"]=None
df["customer_id_has_paid"]=None
df["shop_id_count_paid_orders_90D"]=None

In [10]:
df

,order_id,customer_id,created_date,order_amount,shop_id,order_status,orders_on_customer_id_7D,customer_id_has_paid,shop_id_count_paid_orders_90D
0,ea44f92ee05880ccb9d1eac565c781c6,124e7b421093cb5ae18df8256ea591ca,2020-01-01,1839.20,4716486e2f5489e28a43354320c656a4,paid,None,None,None
1,6ea88eb8ff98a78e07611a4e955a7d46,ba108833b7381247b6d20fea0501b8dc,2020-04-03,2302.25,4716486e2f5489e28a43354320c656a4,paid,None,None,None
2,342e47f90a085011c3afa94475dc61bd,6536928d8ecbd7d731f5fc2aa22573a9,2020-04-12,1120.06,4716486e2f5489e28a43354320c656a4,paid,None,None,None
3,9566725f601cc9d26943b55aaaf1e5b0,8dd13ca4b535b996a775b66f274b0c1f,2020-05-04,565.00,556e232c4549000a0bdd7e9d23860a86,paid,None,None,None
4,27309cda8bf3a5a17cc7d1781ff998bb,8463b19013300bff71bfa8101773ad3d,2020-06-29,2467.00,24758fef3aa92a7b71c759e248557886,paid,None,None,None
...,...,...,...,...,...,...,...,...,...
995,678abfea3e904a321ee8a0457e79189b,42e45fe7bfc70e571261179918805cdb,2022-01-04,73.00,8312706086e548c51cc2cfd9c6a010a1,due,None,None,None
996,70c87363189e507cc7a41abacc2d573a,91565ec5169fac1639aa0160f1340523,2022-01-04,3567.00,f7b3fb70d1663504b4c1ec5b1c095888,due,None,None,None
997,d355bff8791ec22b8a73ce20df148caa,d270ec89c34859033f70604017107709,2022-01-04,65.00,8312706086e548c51cc2cfd9c6a010a1,due,None,None,None
998,92c0338455e56a624cf7bb0b3bdaa400,d7052b67b357eb15ef0429df9ce47105,2022-01-05,402.00,14a206b5298dbfa9454e85293881a249,due,None,None,None


### Calculating `orders_on_customer_id_7D` and `customer_id_has_paid`

**Assumption for calculating `customer_id_has_paid`** <br>
If customer has any single paid order prior to current order in last 7 days, it `customer_id_has_paid` be marked as `True`

In [11]:
groups=df.groupby(df.customer_id)
for customer_id, sub_group in groups:
  for index, row in sub_group.iterrows():
    current_date=row["created_date"]
    days_7=current_date-timedelta(7)
    orders=sub_group[(sub_group.created_date<current_date) & (sub_group.created_date>=days_7)]["order_id"].count()
    df.at[index,'orders_on_customer_id_7D']=orders
    has_paid=sub_group[(sub_group.created_date<current_date) & (sub_group.order_status=='paid')]["order_id"].count()
    df.at[index,'customer_id_has_paid']=(has_paid>0)

### Calculating `shop_id_count_paid_orders_90D`



In [12]:
groups=df.groupby(df.shop_id)
for gid, sub_group in groups:
  for index, row in sub_group.iterrows():
    current_date=row["created_date"]
    days_90=current_date-timedelta(90)
    orders=sub_group[(sub_group.created_date<current_date) & (sub_group.created_date>=days_90)]["order_id"].count()
    df.at[index,'shop_id_count_paid_orders_90D']=orders

In [13]:
df

,order_id,customer_id,created_date,order_amount,shop_id,order_status,orders_on_customer_id_7D,customer_id_has_paid,shop_id_count_paid_orders_90D
0,ea44f92ee05880ccb9d1eac565c781c6,124e7b421093cb5ae18df8256ea591ca,2020-01-01,1839.20,4716486e2f5489e28a43354320c656a4,paid,0,False,0
1,6ea88eb8ff98a78e07611a4e955a7d46,ba108833b7381247b6d20fea0501b8dc,2020-04-03,2302.25,4716486e2f5489e28a43354320c656a4,paid,0,False,0
2,342e47f90a085011c3afa94475dc61bd,6536928d8ecbd7d731f5fc2aa22573a9,2020-04-12,1120.06,4716486e2f5489e28a43354320c656a4,paid,0,False,1
3,9566725f601cc9d26943b55aaaf1e5b0,8dd13ca4b535b996a775b66f274b0c1f,2020-05-04,565.00,556e232c4549000a0bdd7e9d23860a86,paid,0,False,0
4,27309cda8bf3a5a17cc7d1781ff998bb,8463b19013300bff71bfa8101773ad3d,2020-06-29,2467.00,24758fef3aa92a7b71c759e248557886,paid,0,False,0
...,...,...,...,...,...,...,...,...,...
995,678abfea3e904a321ee8a0457e79189b,42e45fe7bfc70e571261179918805cdb,2022-01-04,73.00,8312706086e548c51cc2cfd9c6a010a1,due,0,False,230
996,70c87363189e507cc7a41abacc2d573a,91565ec5169fac1639aa0160f1340523,2022-01-04,3567.00,f7b3fb70d1663504b4c1ec5b1c095888,due,0,False,47
997,d355bff8791ec22b8a73ce20df148caa,d270ec89c34859033f70604017107709,2022-01-04,65.00,8312706086e548c51cc2cfd9c6a010a1,due,0,False,230
998,92c0338455e56a624cf7bb0b3bdaa400,d7052b67b357eb15ef0429df9ce47105,2022-01-05,402.00,14a206b5298dbfa9454e85293881a249,due,0,False,12


In [14]:
df.to_csv("interview_df_solution.csv",index=False)